In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=76b4819b56bb8c3090fe43ff9508c6cae7936e0f21ddca1625b085bb1476b9b0
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName('Practise').getOrCreate()

In [ ]:
spark

In [ ]:
df_pyspark=spark.read.csv('sales_data.csv',header=True,inferSchema=True)

In [ ]:
df_pyspark.show()

+--------------------+----------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+
|              Region|   Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|
+--------------------+----------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+
|Middle East and N...|      NULL|      Cosmetics|      Offline|             M|10/18/2014|686800706|10/31/2014|      8446|     437.2|   263.33|
|       North America|    Canada|     Vegetables|       Online|             M| 11/7/2011|185941302| 12/8/2011|      NULL|    154.06|    90.93|
|Middle East and N...|     Libya|      Baby Food|      Offline|             C|10/31/2016|246222341| 12/9/2016|      1517|      NULL|   159.42|
|                Asia|     Japan|         Cereal|      Offline|             C| 4/10/2010|161442649| 5/12/2010|      3322|     205.7|     NULL|

In [ ]:
df_pyspark=df_pyspark.na.drop(how="any",subset=['Country'])

In [ ]:
df_pyspark.show()

+--------------------+--------------------+----------+-------------+--------------+----------+---------+----------+----------+----------+---------+
|              Region|             Country| Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|
+--------------------+--------------------+----------+-------------+--------------+----------+---------+----------+----------+----------+---------+
|       North America|              Canada|Vegetables|       Online|             M| 11/7/2011|185941302| 12/8/2011|      NULL|    154.06|    90.93|
|Middle East and N...|               Libya| Baby Food|      Offline|             C|10/31/2016|246222341| 12/9/2016|      1517|      NULL|   159.42|
|                Asia|               Japan|    Cereal|      Offline|             C| 4/10/2010|161442649| 5/12/2010|      3322|     205.7|     NULL|
|  Sub-Saharan Africa|                Chad|    Fruits|      Offline|             H| 8/16/2011|645713555| 8/31/20

In [ ]:
from pyspark.ml.feature import Imputer

imputer = Imputer(
    inputCols=['Units Sold','Unit Price', 'Unit Cost'],
    outputCols=["{}_imputed".format(c) for c in ['Units Sold','Unit Price', 'Unit Cost']]
    ).setStrategy("median")

In [ ]:
imputed_df = imputer.fit(df_pyspark).transform(df_pyspark)
imputed_df.show()

+--------------------+--------------------+----------+-------------+--------------+----------+---------+----------+----------+----------+---------+------------------+------------------+-----------------+
|              Region|             Country| Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Units Sold_imputed|Unit Price_imputed|Unit Cost_imputed|
+--------------------+--------------------+----------+-------------+--------------+----------+---------+----------+----------+----------+---------+------------------+------------------+-----------------+
|       North America|              Canada|Vegetables|       Online|             M| 11/7/2011|185941302| 12/8/2011|      NULL|    154.06|    90.93|              5006|            154.06|            90.93|
|Middle East and N...|               Libya| Baby Food|      Offline|             C|10/31/2016|246222341| 12/9/2016|      1517|      NULL|   159.42|              1517|            154.06

In [ ]:
columns_to_drop = ['Units Sold', 'Unit Price', 'Unit Cost']
imputed_df = imputed_df.drop(*columns_to_drop)

imputed_df.show()

+--------------------+--------------------+----------+-------------+--------------+----------+---------+----------+------------------+------------------+-----------------+
|              Region|             Country| Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold_imputed|Unit Price_imputed|Unit Cost_imputed|
+--------------------+--------------------+----------+-------------+--------------+----------+---------+----------+------------------+------------------+-----------------+
|       North America|              Canada|Vegetables|       Online|             M| 11/7/2011|185941302| 12/8/2011|              5006|            154.06|            90.93|
|Middle East and N...|               Libya| Baby Food|      Offline|             C|10/31/2016|246222341| 12/9/2016|              1517|            154.06|           159.42|
|                Asia|               Japan|    Cereal|      Offline|             C| 4/10/2010|161442649| 5/12/2010|              3322|      

In [ ]:
from pyspark.sql.functions import col, format_number,sum,lit,round
from pyspark.sql.types import DoubleType

df = imputed_df.withColumn("Total_Cost", col("Units Sold_imputed") * col("Unit Cost_imputed"))
df = df.withColumn("Total_Cost", round(col("Total_Cost"), 2))

In [ ]:
df.show()

+--------------------+--------------------+----------+-------------+--------------+----------+---------+----------+------------------+------------------+-----------------+----------+
|              Region|             Country| Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold_imputed|Unit Price_imputed|Unit Cost_imputed|Total_Cost|
+--------------------+--------------------+----------+-------------+--------------+----------+---------+----------+------------------+------------------+-----------------+----------+
|       North America|              Canada|Vegetables|       Online|             M| 11/7/2011|185941302| 12/8/2011|              5006|            154.06|            90.93| 455195.58|
|Middle East and N...|               Libya| Baby Food|      Offline|             C|10/31/2016|246222341| 12/9/2016|              1517|            154.06|           159.42| 241840.14|
|                Asia|               Japan|    Cereal|      Offline|             C| 4

In [ ]:
df = df.withColumn("Total_Revenue", col("Units Sold_imputed") * col("Unit Price_imputed"))
df = df.withColumn("Total_Revenue", round(col("Total_Revenue"), 2))

df = df.withColumn("Total_Profit", col("Total_Revenue") - col("Total_Cost"))
df = df.withColumn("Total_Profit", round(col("Total_Profit"), 2))

In [ ]:
df.show()

+--------------------+--------------------+----------+-------------+--------------+----------+---------+----------+------------------+------------------+-----------------+----------+-------------+------------+
|              Region|             Country| Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold_imputed|Unit Price_imputed|Unit Cost_imputed|Total_Cost|Total_Revenue|Total_Profit|
+--------------------+--------------------+----------+-------------+--------------+----------+---------+----------+------------------+------------------+-----------------+----------+-------------+------------+
|       North America|              Canada|Vegetables|       Online|             M| 11/7/2011|185941302| 12/8/2011|              5006|            154.06|            90.93| 455195.58|    771224.36|   316028.78|
|Middle East and N...|               Libya| Baby Food|      Offline|             C|10/31/2016|246222341| 12/9/2016|              1517|            154.06|       

In [ ]:
total_cost_by_product = df.groupBy("Item Type") \
    .agg(format_number(sum(col("Total_Cost")), 2).alias("Total_Total_Cost")) \
    .orderBy("Item Type")

total_cost_by_product.show()

+---------------+----------------+
|      Item Type|Total_Total_Cost|
+---------------+----------------+
|      Baby Food|   51,130,537.14|
|      Beverages|   17,953,976.03|
|         Cereal|   40,170,688.60|
|        Clothes|   16,620,121.28|
|      Cosmetics|   71,699,199.36|
|         Fruits|    5,146,107.12|
|      Household|  127,663,742.00|
|           Meat|  100,319,529.75|
|Office Supplies|  136,188,375.36|
|  Personal Care|   22,510,292.58|
|         Snacks|   31,457,919.36|
|     Vegetables|   32,191,724.46|
+---------------+----------------+



In [ ]:
total_Revenue_by_product = df.groupBy("Item Type") \
    .agg(format_number(sum(col("Total_Revenue")), 2).alias("Total_Revenue_Cost")) \
    .orderBy("Item Type")

total_Revenue_by_product.show()

+---------------+------------------+
|      Item Type|Total_Revenue_Cost|
+---------------+------------------+
|      Baby Food|     80,565,795.50|
|      Beverages|     26,519,615.03|
|         Cereal|     68,180,278.24|
|        Clothes|     37,218,046.20|
|      Cosmetics|    122,128,032.54|
|         Fruits|     12,514,633.96|
|      Household|    157,558,416.93|
|           Meat|    124,952,771.15|
|Office Supplies|    181,633,529.92|
|  Personal Care|     33,603,770.64|
|         Snacks|     49,356,008.36|
|     Vegetables|     53,922,540.60|
+---------------+------------------+



In [ ]:
total_Profit_by_product = df.groupBy("Item Type") \
    .agg(format_number(sum(col("Total_Profit")), 2).alias("Total_Profit_Cost")) \
    .orderBy("Item Type")

total_Profit_by_product.show()

+---------------+-----------------+
|      Item Type|Total_Profit_Cost|
+---------------+-----------------+
|      Baby Food|    29,435,258.36|
|      Beverages|     8,565,639.00|
|         Cereal|    28,009,589.64|
|        Clothes|    20,597,924.92|
|      Cosmetics|    50,428,833.18|
|         Fruits|     7,368,526.84|
|      Household|    29,894,674.93|
|           Meat|    24,633,241.40|
|Office Supplies|    45,445,154.56|
|  Personal Care|    11,093,478.06|
|         Snacks|    17,898,089.00|
|     Vegetables|    21,730,816.14|
+---------------+-----------------+



In [ ]:
df.write.csv("imputed_data.csv", header=True)

In [ ]:
spark.stop()